In [1]:
seed = 3906303

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pickle
import glob, os
import gzip
import random
np.random.seed(seed)
random.seed(seed)

## Load dataset

Here we read in all the measurement files created in the datagen notebook. Each dataframe is a separate file, which is each a separate table of measurement readings of particular types.

In [3]:
# Create dictionary of measurement-type dataframes
MEAS = {
    name: pd.read_csv(
        fp,
        parse_dates=['charttime'],       # parsed dates on the fly
        low_memory=False                 # avoids mixed-type warnings
    )
    for fp  in sorted(glob.glob('Datagen/*_filtered_chartevents.csv'))
    for name in [os.path.basename(fp).split('_')[0]]
}

FEATURES = list(MEAS.keys())
F        = len(FEATURES)             # number of features
GLU_IDX  = FEATURES.index('bg')     # which column contains glucose

In [4]:
FEATURES

['abpm', 'bg', 'cvp', 'f', 'hr', 'hrh', 'hrl', 'nbpm', 'rr', 'spo2']

# Data filtering and cleaning

Below we filter impossible values for the physiological features. No negative numbers, except for rare cases of CVP feature.

In [5]:
#  Per-variable rules. Only reject impossible numbers
LOGIC_RULES = {
    'bg' : lambda s: (s >= 10) & (s <= 2656),         # historically recorded extreme mg/dL
    'hr'  : lambda s: s >= 0,                     # beats / min
    'rr'  : lambda s: s >= 0,                     # breaths / min
    'spo2': lambda s: (s >= 0) & (s <= 100),      # % saturation
    'nbpm': lambda s: s >= 0,                     # mm Hg
    'abpm': lambda s: s >= 0,                     # mm Hg
    'f'   : lambda s: s >= 0,                     # F < 0 impossible
    # CVP can dip a few mm Hg negative during inspiration
    'cvp' : lambda s: s >= -5,                    # allow mild physio negative
    'hrl' : lambda s: s >= 0,
    'hrh' : lambda s: s >= 0,
}

def filter_impossible(df, short_name):
    df = df.copy()
    rule = LOGIC_RULES.get(short_name)
    if rule is None:
        raise KeyError(f"No logic rule defined for '{short_name}'")
    return df[rule(df['value'])]

# Apply to every dataframe in the MEAS dict
MEAS_CLEAN = {
    name: filter_impossible(df, name)
    for name, df in MEAS.items()
}

# sanity check
for name, df in MEAS_CLEAN.items():
    bad = df['value'].isna().sum()
    print(f"{name}: kept {len(df):,} rows  (dropped {bad:,} NaNs / impossible)")


abpm: kept 3,094,749 rows  (dropped 0 NaNs / impossible)
bg: kept 1,814,111 rows  (dropped 0 NaNs / impossible)
cvp: kept 992,340 rows  (dropped 0 NaNs / impossible)
f: kept 2,055,038 rows  (dropped 0 NaNs / impossible)
hr: kept 8,752,066 rows  (dropped 0 NaNs / impossible)
hrh: kept 843,381 rows  (dropped 0 NaNs / impossible)
hrl: kept 843,692 rows  (dropped 0 NaNs / impossible)
nbpm: kept 5,372,920 rows  (dropped 0 NaNs / impossible)
rr: kept 8,636,655 rows  (dropped 0 NaNs / impossible)
spo2: kept 8,566,844 rows  (dropped 0 NaNs / impossible)


# Split Dataset into Training, Validation and Testing Datasets

In [6]:
unique_subjects = MEAS['bg']['subject_id'].unique()
train_subjects, test_subjects = train_test_split(unique_subjects, test_size=0.2,
                                         random_state=seed)
train_subjects, val_subjects  = train_test_split(train_subjects, test_size=0.25,
                                         random_state=seed)

SPLIT = {'train': train_subjects, 'val': val_subjects, 'test': test_subjects}

### Hyperparameters

In [7]:
FREQ       = '2H'           # 2-hour bins
MAX_HOURS  = 48
STEPS      = MAX_HOURS // 2 # 48 h ÷ 2 h = 24 bins
L          = 6               # look-back  steps (12 hours of history)
H          = 1               # look-ahead steps (direct subsequent prediction)
SENTINEL   = -999.0            # value the Masking layer will ignore

### Here we resample the data to ensure there is a row for every timestep interval of 2 hours.

In [8]:
def resample_stay_multi(stay_id, dfs_by_feat):

    mat = np.full((STEPS, F), np.nan, dtype='float32')

    # Find earliest timestamp amongst all features
    start_time = None
    # Visit all feature dataframes one by one, and keep track of earliest recorded time
    for df in dfs_by_feat.values():
        s = df.loc[df['stay_id'] == stay_id, 'charttime']
        if not s.empty:
            t0 = s.min()
            start_time = t0 if start_time is None else min(start_time, t0)

    # Return None if none of the feature dfs have the stay_id
    if start_time is None:
        return None

    # Standardise horizon length of icu stay measurements
    horizon = pd.date_range(start=start_time.floor(FREQ),
                            periods=STEPS,
                            freq=FREQ)

    # Fill one column of matrix per feature
    for j, feat in enumerate(FEATURES):
        g = dfs_by_feat[feat]
        g = g[g['stay_id'] == stay_id]
        if g.empty:
          # Column remains NaNs
            continue
        s = (g.set_index('charttime')['value']
               .resample(FREQ).mean())
        # reindex sets length of values to be exactly STEPS length long,
        # by trancating or padding with NaNs
        mat[:, j] = s.reindex(horizon).values

    return mat                       # (STEPS, F)

Here we create the data to be fed into the model. Each row of data in the X datasets is a sequence of BG measurements. The Y datasets contain a BG reading that follows directly from that sequence after.

The approach we took is so that a sequence of measurements (starting from the first) is created of certain length from each ICU admission, which is taken as the X, and the following value after the sequence is taken as the Y. Then the window is slid over by one so that the sequence is slightly different, creating a new X value and a new following Y value. This process is repeated until the end of the ICU admission measurement sequence is reached, then a new ICU admission is chosen, until all ICU admissions have had their sequences recorded.

In [9]:
def make_Xy_multi(subj_ids, fit_scaler=False, scaler_dict=None):
    X_stays = []

    # Create dictionary of all feature dataframes for the current split
    dfs_by_feat = {}
    for feat, df in MEAS.items():
        df_ = df[df['subject_id'].isin(subj_ids)].copy()
        df_['charttime'] = pd.to_datetime(df_['charttime'])
        dfs_by_feat[feat] = df_

    # Retrieve all unique stay_ids across all feature dataframes
    all_stays = pd.concat([d[['stay_id']] for d in dfs_by_feat.values()]
                          ).drop_duplicates()['stay_id'].values
    # Creates a list of (STEPS, F) matrices
    for sid in all_stays:
        M = resample_stay_multi(sid, dfs_by_feat)   # (STEPS, F) or None
        # Only append the matrix if at least one of the dataframes contains a value
        if M is not None:
            X_stays.append(M)

    # Convert to list of stays to 3D tensor shape
    X_stays = np.stack(X_stays)          # (N_stays, STEPS, F)

    # Feature-wise Standardisation
    if scaler_dict is None:
        scaler_dict = {}
    for j in range(F):
        # Concatenate all feature j columns across all stays, include every row, stack them
        col = X_stays[:, :, j].ravel()
        # NaN mask
        mask = ~np.isnan(col)
        if fit_scaler:
            mu = np.nanmean(col)
            sd = np.nanstd(col)
            # Protects program from division-by-zero error by ensuring if col is all NaNs, 0 is replaced with 1
            sd = sd if sd != 0 else 1.0
            scaler_dict[j] = (mu, sd)
        # Retrieves values for std and mean for when non-training splits are scaled
        mu, sd = scaler_dict[j]
        # Scales non-NaN data
        col[mask] = (col[mask] - mu) / sd

        # Sets a slice of dimension (X_stays.shape[0], STEPS) of a particular feature to the col variable
        X_stays[:, :, j] = col.reshape(X_stays.shape[0], STEPS)

    # Sliding window
    windows, targets = [], []
    for stay in X_stays:
        # Window slides across every point so that the entire horizon of the ICU stay is included once
        for t0 in range(0, STEPS - L - H + 1):
            x_win = stay[t0 : t0 + L, :]                # Look-back window
            y_val = stay[t0 + L + H - 1, GLU_IDX]       # Glucose value following window
            if not np.isnan(y_val):                     # ensure that the final y value is an actual value
                windows.append(x_win)
                targets.append(y_val)

    X = np.array(windows, dtype='float32')             # (N, L, F)
    y = np.array(targets, dtype='float32')             # (N,)

    # mask NaN timesteps
    X[np.isnan(X)] = SENTINEL
    return X, y, scaler_dict

In [10]:
os.makedirs('Pickles', exist_ok=True)

In [11]:
X_train, y_train, SCALERS = make_Xy_multi(SPLIT['train'], fit_scaler=True)

# Save train data and scalers immediately
train_path = 'Pickles/glu_train.pkl.gz'
with gzip.open(train_path, 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'y_train': y_train,
        'scalers': SCALERS
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved train set -> {os.path.getsize(train_path)/1e6:.1f} MB")

Saved train set -> 10.1 MB


In [12]:
X_val, y_val, _ = make_Xy_multi(SPLIT['val'], scaler_dict=SCALERS)

# Save val data immediately
val_path = 'Pickles/glu_val.pkl.gz'
with gzip.open(val_path, 'wb') as f:
    pickle.dump({
        'X_val': X_val,
        'y_val': y_val
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved validation set -> {os.path.getsize(val_path)/1e6:.1f} MB")


Saved validation set -> 3.4 MB


In [13]:
X_test, y_test, _ = make_Xy_multi(SPLIT['test'], scaler_dict=SCALERS)

# Save test data immediately
test_path = 'Pickles/glu_test.pkl.gz'
with gzip.open(test_path, 'wb') as f:
    pickle.dump({
        'X_test': X_test,
        'y_test': y_test
    }, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved test set -> {os.path.getsize(test_path)/1e6:.1f} MB")

Saved test set -> 3.4 MB


# Load saved tensors

In [11]:
train_path = 'Pickles/glu_train.pkl.gz'

with gzip.open(train_path, 'rb') as f:
    train_bundle = pickle.load(f)

X_train = train_bundle['X_train']
y_train = train_bundle['y_train']
SCALERS = train_bundle['scalers']

In [ ]:
val_path = 'Pickles/glu_val.pkl.gz'

with gzip.open(val_path, 'rb') as f:
    val_bundle = pickle.load(f)

X_val = val_bundle['X_val']
y_val = val_bundle['y_val']

In [ ]:
test_path = 'Pickles/glu_test.pkl.gz'

with gzip.open(test_path, 'rb') as f:
    test_bundle = pickle.load(f)

X_test = test_bundle['X_test']
y_test = test_bundle['y_test']

In [ ]:
X_train, y_train, SCALERS = make_Xy_multi(SPLIT['train'], fit_scaler=True)
X_val,   y_val,  _        = make_Xy_multi(SPLIT['val'],   scaler_dict=SCALERS)
X_test,  y_test, _        = make_Xy_multi(SPLIT['test'],  scaler_dict=SCALERS)

<ipython-input-9-7a0679aeab02>:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(start=start_time.floor(FREQ),
<ipython-input-9-7a0679aeab02>:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(FREQ).mean())
<ipython-input-9-7a0679aeab02>:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horizon = pd.date_range(start=start_time.floor(FREQ),
<ipython-input-9-7a0679aeab02>:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(FREQ).mean())


In [14]:
# After you build SCALERS in make_Xy_multi(...)
mu_glu, std_glu = SCALERS[GLU_IDX]          # μ, σ learned by StandardScaler
print(f"Glucose  μ={mu_glu:.1f}  σ={std_glu:.1f}")


Glucose  μ=246.7  σ=9512.7


In [15]:
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape,   y_val.shape)
print("Test :", X_test.shape,  y_test.shape)

Train: (255182, 6, 10) (255182,)
Val  : (85902, 6, 10) (85902,)
Test : (84769, 6, 10) (84769,)


In [16]:
print("Any NaN in y_train ?", np.isnan(y_train).any())
print("Any NaN in X_train ?", np.isnan(X_train).any())

# How many?
print("NaN count:", np.isnan(X_train).sum())

Any NaN in y_train ? False
Any NaN in X_train ? False
NaN count: 0


In [17]:
BATCH = 64
def make_ds(X, y, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(len(y), seed=seed)
    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

In [18]:
ds_train = make_ds(X_train, y_train, shuffle=True)
ds_val   = make_ds(X_val,   y_val,   shuffle=False)
ds_test  = make_ds(X_test,  y_test,  shuffle=False)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=SENTINEL, input_shape=(L, F)),
    tf.keras.layers.LSTM(64),
    # A dense layer here is necessary, as each hidden state output, which is
    # the output of an LSTM unit is only within the range of a tanh activation function,
    # so further transformation is needed
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)                   # regression
])

model.compile(loss='mse', optimizer='adam', metrics=['mae', RootMeanSquaredError(name='rmse'), 'mse'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 6, 10)             0         
                                                                 
 lstm (LSTM)                 (None, 64)                19200     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 21,313
Trainable params: 21,313
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Create directory if it doesn't exist
os.makedirs('models', exist_ok=True)

In [20]:
#  Create a checkpoint callback
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/multivar_epoch_{epoch:02d}.h5',
    save_freq='epoch',                        # save after each epoch
    save_weights_only=False,                  # save the entire model (architecture + weights + optimizer state)
    verbose=1)

history = model.fit(
    ds_train,
    epochs=100,
    # Validation is used to determine if model is overfitting
    validation_data=ds_val,
    callbacks=[ckpt],
    verbose=1
)

results = model.evaluate(ds_test, return_dict=True)
print(f"Test RMSE = {results['rmse']:.2f}   |   MAE = {results['mae']:.2f}")

Epoch 1/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6996 - mae: 0.0349 - rmse: 0.8364 - mse: 0.6996
Epoch 1: saving model to models\multivar_epoch_01.h5
3988/3988 [==============================] - 18s 4ms/step - loss: 0.6994 - mae: 0.0349 - rmse: 0.8363 - mse: 0.6994 - val_loss: 0.3442 - val_mae: 0.0163 - val_rmse: 0.5867 - val_mse: 0.3442
Epoch 2/100
3978/3988 [============================>.] - ETA: 0s - loss: 0.6995 - mae: 0.0252 - rmse: 0.8364 - mse: 0.6995
Epoch 2: saving model to models\multivar_epoch_02.h5
3988/3988 [==============================] - 16s 4ms/step - loss: 0.6979 - mae: 0.0252 - rmse: 0.8354 - mse: 0.6979 - val_loss: 0.3440 - val_mae: 0.0148 - val_rmse: 0.5865 - val_mse: 0.3440
Epoch 3/100
3985/3988 [============================>.] - ETA: 0s - loss: 0.6980 - mae: 0.0229 - rmse: 0.8355 - mse: 0.6980
Epoch 3: saving model to models\multivar_epoch_03.h5
3988/3988 [==============================] - 17s 4ms/step - loss: 0.6976 - mae: 0.0229 - rms

3982/3988 [============================>.] - ETA: 0s - loss: 0.6981 - mae: 0.0168 - rmse: 0.8355 - mse: 0.6981
Epoch 23: saving model to models\multivar_epoch_23.h5
3988/3988 [==============================] - 31s 8ms/step - loss: 0.6972 - mae: 0.0168 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0087 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 24/100
3985/3988 [============================>.] - ETA: 0s - loss: 0.6976 - mae: 0.0165 - rmse: 0.8352 - mse: 0.6976
Epoch 24: saving model to models\multivar_epoch_24.h5
3988/3988 [==============================] - 31s 8ms/step - loss: 0.6972 - mae: 0.0165 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0099 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 25/100
3984/3988 [============================>.] - ETA: 0s - loss: 0.6978 - mae: 0.0165 - rmse: 0.8353 - mse: 0.6978
Epoch 25: saving model to models\multivar_epoch_25.h5
3988/3988 [==============================] - 33s 8ms/step - loss: 0.6972 - mae: 0.0164 - rmse: 0.83

3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0169 - rmse: 0.8351 - mse: 0.6974
Epoch 45: saving model to models\multivar_epoch_45.h5
3988/3988 [==============================] - 40s 10ms/step - loss: 0.6972 - mae: 0.0169 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0128 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 46/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0170 - rmse: 0.8351 - mse: 0.6974
Epoch 46: saving model to models\multivar_epoch_46.h5
3988/3988 [==============================] - 37s 9ms/step - loss: 0.6972 - mae: 0.0170 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0092 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 47/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0163 - rmse: 0.8351 - mse: 0.6974
Epoch 47: saving model to models\multivar_epoch_47.h5
3988/3988 [==============================] - 44s 11ms/step - loss: 0.6972 - mae: 0.0163 - rmse: 0.

3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0170 - rmse: 0.8351 - mse: 0.6974
Epoch 67: saving model to models\multivar_epoch_67.h5
3988/3988 [==============================] - 58s 15ms/step - loss: 0.6972 - mae: 0.0170 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0104 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 68/100
3984/3988 [============================>.] - ETA: 0s - loss: 0.6978 - mae: 0.0164 - rmse: 0.8353 - mse: 0.6978
Epoch 68: saving model to models\multivar_epoch_68.h5
3988/3988 [==============================] - 55s 14ms/step - loss: 0.6972 - mae: 0.0164 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0094 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 69/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0159 - rmse: 0.8351 - mse: 0.6974
Epoch 69: saving model to models\multivar_epoch_69.h5
3988/3988 [==============================] - 61s 15ms/step - loss: 0.6972 - mae: 0.0160 - rmse: 0

3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0166 - rmse: 0.8351 - mse: 0.6974
Epoch 89: saving model to models\multivar_epoch_89.h5
3988/3988 [==============================] - 60s 15ms/step - loss: 0.6972 - mae: 0.0166 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3439 - val_mae: 0.0099 - val_rmse: 0.5864 - val_mse: 0.3439
Epoch 90/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0165 - rmse: 0.8351 - mse: 0.6974
Epoch 90: saving model to models\multivar_epoch_90.h5
3988/3988 [==============================] - 60s 15ms/step - loss: 0.6972 - mae: 0.0165 - rmse: 0.8350 - mse: 0.6972 - val_loss: 0.3440 - val_mae: 0.0210 - val_rmse: 0.5865 - val_mse: 0.3440
Epoch 91/100
3986/3988 [============================>.] - ETA: 0s - loss: 0.6974 - mae: 0.0173 - rmse: 0.8351 - mse: 0.6974
Epoch 91: saving model to models\multivar_epoch_91.h5
3988/3988 [==============================] - 61s 15ms/step - loss: 0.6972 - mae: 0.0173 - rmse: 0

ValueError: too many values to unpack (expected 2)

In [23]:
results = model.evaluate(ds_test, return_dict=True)
print(f"Test RMSE = {results['rmse']:.2f}   |   MAE = {results['mae']:.2f}")

1325/1325 [==============================] - 3s 2ms/step - loss: 0.2607 - mae: 0.0070 - rmse: 0.5106 - mse: 0.2607
Test RMSE = 0.51   |   MAE = 0.01


In [21]:
y_pred_z = model.predict(ds_test, verbose=0).ravel()   # z-scores
y_true_z = y_test

# inverse-transform back to mg/dL
y_pred = y_pred_z * std_glu + mu_glu
y_true = y_true_z * std_glu + mu_glu

# metrics in true units
mse  = np.mean((y_true - y_pred) ** 2)    
rmse = np.sqrt(mse)
mae  = np.mean(np.abs(y_true - y_pred))

print(f"Test MSE  = {mse :.2f} (mg/dL)^2")
print(f"Test RMSE = {rmse:.2f} mg/dL")
print(f"Test MAE  = {mae :.2f} mg/dL")

Test MSE  = 23590026.00 (mg/dL)^2
Test RMSE = 4856.96 mg/dL
Test MAE  = 66.28 mg/dL
